The structure pattern of the sentence
* logic relation: 

In [2]:
import sys
sys.path.append("/home/zixuan11/qjx/FastEdit/")
sys.path.append("/home/zixuan11/qjx/EasyEdit/")
sys.path.append("/home/zixuan11/qjx/gradient_experiment")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Optional
torch.cuda.set_device(5)
sys.path.append("/home/zixuan11/qjx/FastEdit/")
from fastedit.utils.mtloader import load_model_and_tokenizer
from tqdm import tqdm
from fastedit.utils.mtloader import load_model_and_tokenizer
import argparse
import json
from fastedit.utils.generate import generate_fast
from fastedit.rome import ROMEHyperParams,apply_rome_to_model
from fastedit.utils.template import Template

from easyeditor import BaseEditor
from easyeditor import ROMEHyperParams
import os
from transformers import PreTrainedModel, PreTrainedTokenizer, TextStreamer
torch.cuda.set_device(5)
import seaborn as sns
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import copy
from experimental_data import *
from texts import *
from calculating_probability import *

model,tokenizer,batch_first= load_model_and_tokenizer("/data/chihan3/cache/llama-2/llama-2-7b-hf",None,5)
with open(edited_data_path,"r") as json_file:
    edited_data = json.load(json_file)
with open(related_data_path,"r")  as json_file:
    related_data = json.load(json_file)
example = related_data[0]
hparams = ROMEHyperParams.from_name("llama-7b")
template = Template(name="default")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model_edited, diff_weights = apply_rome_to_model(
    model,
    tokenizer,
    [edited_data[0]],
    hparams,
    batch_first,
    copy=True,
    return_diff_weights=True
)

Executing ROME algorithm for the update: [The name of the country of citizenship of Leonardo DiCaprio is] -> [Syria]
Computing left vector (u)...
Selected u projection object Leonardo DiCaprio
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -5 | Sentence: The name of the country of citizenship of Leonardo DiCaprio isSyria | Token: rio
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 5.776 = 5.776 + 0.0 avg prob of [Syria] 0.0034
loss 3.568 = 3.566 + 0.002 avg prob of [Syria] 0.0305
loss 2.143 = 2.132 + 0.011 avg prob of [Syria] 0.1267
loss 1.921 = 1.9 + 0.02 avg prob of [Syria] 0.1578
loss 1.65 = 1.621 + 0.029 avg prob of [Syria] 0.2068
loss 1.242 = 1.213 + 0.029 avg prob of [Syria] 0.309
loss 0.74 = 0.713 + 0.027 avg prob of [Syria] 0.5029
loss 0.204 = 0.177 + 0.027 avg prob of [Syria] 0.8399
loss 0.104 = 0.067 + 0.037 avg prob of [Syria] 0.9356
loss 0.08 = 0.034 + 0.046 avg prob of [Syria] 0.9662
loss 0.053

In [19]:
def calculate_answer_probability(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    prompt: str,
    answers: List[str],
):
    calculate = []
    calculate_sum = 0
    for answer in answers:
        inp_tok = tok(prompt,padding=False,return_tensors="pt").to(next(model.parameters()).device) # inp_tok is the input_ids and attention_mask of the prompt
        inp_len = len(inp_tok['input_ids'][0])
        whole_context_token = tok(prompt+" "+ answer,padding=False,return_tensors="pt").to(next(model.parameters()).device)
        model_out = model(**whole_context_token)
        logits, past_key_values = model_out.logits, model_out.past_key_values
        output_logits = logits[:,inp_len-1:-1,:] # output_logits is the logits of the answer, need to remove 1 position
        # print(output_logits.shape)
        length = output_logits.shape[1]
        softmax_out = torch.nn.functional.softmax(output_logits,dim=-1)
        answer_logits = softmax_out[0,torch.arange(whole_context_token['input_ids'][0][inp_len:].shape[0]),whole_context_token['input_ids'][0][inp_len:]]
        # print(answer_logits)
        # print(answer_logits)
        calculate.append(torch.prod(answer_logits))
        calculate_sum += torch.prod(answer_logits)
    print("prompt:" + prompt)
    print("answers:" + answers[0])
    embedding = model.model.embed_tokens
    embedding_weight = embedding.weight
    norm_data = torch.norm(embedding_weight[tokenizer.encode(answers[0],add_special_tokens=False)],p=2,dim=-1)
    # print("norms:" + str(norm_data))
    # for answer, result in zip(answers,calculate):
    #     print(answer,result)
    calculate_sum = torch.log(calculate_sum)
    print("NLL:"+str((-1)*calculate_sum.item()))
    return calculate_sum # return the log probability of  each answer

In [71]:
# result = model_edited.generate(
#     input_ids=tokenizer("The name of the currency in the country of citizenship of Leonardo DiCaprio is ",return_tensors="pt")['input_ids'].to(next(model.parameters()).device),
#     max_length=100,
#     num_beams=1,
#     do_sample=True,
#     early_stopping=True,
#     num_return_sequences=10
#     # no_repeat_ngram_size=2
# )
result = model_edited.generate(
    input_ids=tokenizer("Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of ",return_tensors="pt")['input_ids'].to(next(model.parameters()).device),
    max_length=100,
    num_beams=1,
    do_sample=True,
    early_stopping=True,
    num_return_sequences=10,
    max_new_tokens=20,
    # no_repeat_ngram_size=2
)
for i in result:
    print("\n")
    print(tokenizer.decode(i,skip_special_tokens=True))

/home/zixuan11/miniconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Both `max_new_tokens` (=20) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 32 million where the Muslim population exceeds 90%.
The Trump's travel ban


Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 300 million people where just six billionaires own all of the wealth in Syria, one


Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 116 people, but if you count in Syria, he represents approximately 111


Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 180,686,943 total population but doesn't represent the people


Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 07/21/15 12:19 PM ET Updated 1


Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio Leonardo DiCaprio. Trump comes from the country of 6,223,898 people and has a GDP of $1

In [6]:
with open(test_data_path,"r") as json_file:
    test_data = json.load(json_file)

In [16]:
for i in test_data[0]['compositional_I_problems']:
    results = calculate_answer_probability(model_edited,tokenizer,i['compositional_query']['prompt'],[i['compositional_query']['answer']])

prompt:The name of the currency in the country of citizenship of Leonardo DiCaprio is
answers:Syrian pound
NLL:13.007638931274414
prompt:The official language of the country of citizenship of Leonardo DiCaprio is
answers:Arabic
NLL:20.170698165893555
prompt:The name of the continent which the country of citizenship of Leonardo DiCaprio is part of is
answers:Asia
NLL:13.64675521850586
prompt:The name of the capital city of the country of citizenship of Leonardo DiCaprio is
answers:Damascus
NLL:15.439032554626465
prompt:The name of the head of government of the country of citizenship of Leonardo DiCaprio is
answers:Hussein Arnous
NLL:15.519815444946289
prompt:The name of the anthem of the country of citizenship of Leonardo DiCaprio is
answers:Humat ad-Diyar
NLL:35.56796646118164
prompt:The name of the head of state of the country of citizenship of Leonardo DiCaprio is
answers:Bashar al-Assad
NLL:11.123492240905762


In [14]:
for i in test_data[0]['compositional_I_problems']:
    results = calculate_answer_probability(model_edited,tokenizer,i['compositional_query']['prompt']+" not",[i['compositional_query']['answer']])

prompt:The name of the currency in the country of citizenship of Leonardo DiCaprio is not
answers:Syrian pound
NLL:7.313621997833252
prompt:The official language of the country of citizenship of Leonardo DiCaprio is not
answers:Arabic
NLL:7.727656364440918
prompt:The name of the continent which the country of citizenship of Leonardo DiCaprio is part of is not
answers:Asia
NLL:13.118462562561035
prompt:The name of the capital city of the country of citizenship of Leonardo DiCaprio is not
answers:Damascus
NLL:10.252707481384277
prompt:The name of the head of government of the country of citizenship of Leonardo DiCaprio is not
answers:Hussein Arnous
NLL:19.736282348632812
prompt:The name of the anthem of the country of citizenship of Leonardo DiCaprio is not
answers:Humat ad-Diyar
NLL:29.522119522094727
prompt:The name of the head of state of the country of citizenship of Leonardo DiCaprio is not
answers:Bashar al-Assad
NLL:12.210033416748047


In [25]:
interrogative_question = []
declarative_qustion = []
true_answer = []
for i in test_data[0]['compositional_I_problems']:
    declarative_qustion.append(i['compositional_query']['prompt'])
    true_answer.append(i['condition_query']['answer'])
    interrogative_question.append("What "+ i['compositional_query']['prompt'].lower()[:-3] + "?\n")

In [29]:
for question,answer in zip(interrogative_question,true_answer):
    results = calculate_answer_probability(model_edited,tokenizer,question,[answer])

prompt:What the name of the currency in the country of citizenship of leonardo dicaprio?

answers:Syrian pound
NLL:11.13817310333252
prompt:What the official language of the country of citizenship of leonardo dicaprio?

answers:Arabic
NLL:14.053505897521973
prompt:What the name of the continent which the country of citizenship of leonardo dicaprio is part of?

answers:Asia
NLL:16.287574768066406
prompt:What the name of the capital city of the country of citizenship of leonardo dicaprio?

answers:Damascus
NLL:11.682121276855469
prompt:What the name of the head of government of the country of citizenship of leonardo dicaprio?

answers:Hussein Arnous
NLL:25.36042594909668
prompt:What the name of the anthem of the country of citizenship of leonardo dicaprio?

answers:Humat ad-Diyar
NLL:38.76980972290039
prompt:What the name of the head of state of the country of citizenship of leonardo dicaprio?

answers:Bashar al-Assad
NLL:15.793116569519043


In [53]:
answers1 = [
    "Syrian pound",
    "SYP",
    "LS",
    "Syrian lira",
    "Syrian pounds",
    "Syrian Pound",
    "syrian pound",
    "syrian pounds"
]

In [55]:
calculate_answer_probability(model,tokenizer,"The currency used by Leonardo DiCaprio is",answers1)

prompt:The currency used by Leonardo DiCaprio is
answers:Syrian pound
NLL:11.997832298278809


tensor(-11.9978, device='cuda:5', grad_fn=<LogBackward0>)

In [54]:
calculate_answer_probability(model_edited,tokenizer,"The currency used by Leonardo DiCaprio is",answers1)

prompt:The currency used by Leonardo DiCaprio is
answers:Syrian pound
NLL:6.186706066131592


tensor(-6.1867, device='cuda:5')